<h1>Stacked Auto Encoders Training 6-15-2016</h1>

Assuming CNN-H1, and the 6 CNNs for the patches have been trained, we can use the weights to get activations for their fc6 layer and train 3 layers of our SAE individually.

In [1]:
%load_ext autoreload

In [98]:
%autoreload 2
%matplotlib inline

import os
import numpy as np
import matplotlib.pyplot as plt

from multiprocessing import Process
from scipy.ndimage.filters import gaussian_filter

from skimage import io
from skimage.color import rgb2grey
from skimage.transform import resize

from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras.optimizers import SGD

# from keras.utils import np_utils

np.random.seed(123456)

In [3]:
cnn_h1_weight_file = 'CNN-H1_epoch-20_activations'
cnn_p1_weight_file = 'CNN-P1_epoch-20_activations'
cnn_p2_weight_file = 'CNN-P2_epoch-20_activations'
cnn_p3_weight_file = 'CNN-P3_epoch-20_activations'
cnn_p4_weight_file = 'CNN-P4_epoch-20_activations'
cnn_p5_weight_file = 'CNN-P5_epoch-20_activations'
cnn_p6_weight_file = 'CNN-P6_epoch-20_activations'
y_train_file = 'y_train_activations'

layer_1_size = 2048
layer_2_size = 1024
layer_3_size = 512


In [4]:
def load_activations(file_name):
    return np.load('models/' + file_name + '.npy')

In [5]:
cnn_h1_activations = load_activations(cnn_h1_weight_file)
cnn_p1_activations = load_activations(cnn_p1_weight_file)
cnn_p2_activations = load_activations(cnn_p2_weight_file)
cnn_p3_activations = load_activations(cnn_p3_weight_file)
cnn_p4_activations = load_activations(cnn_p4_weight_file)
cnn_p5_activations = load_activations(cnn_p5_weight_file)
cnn_p6_activations = load_activations(cnn_p6_weight_file)
y_train = load_activations(y_train_file)

In [6]:
def fix_activations(_activations):
    return np.array([activation for activations in _activations for activation in activations])

In [7]:
cnn_h1_activations = fix_activations(cnn_h1_activations)
cnn_p1_activations = fix_activations(cnn_p1_activations)
cnn_p2_activations = fix_activations(cnn_p2_activations)
cnn_p3_activations = fix_activations(cnn_p3_activations)
cnn_p4_activations = fix_activations(cnn_p4_activations)
cnn_p5_activations = fix_activations(cnn_p5_activations)
cnn_p6_activations = fix_activations(cnn_p6_activations)

In [8]:
cnn_h1_activations.shape

(26668, 512)

In [9]:
activations = np.concatenate((
        cnn_h1_activations, 
        cnn_p1_activations,
        cnn_p2_activations,
        cnn_p3_activations,
        cnn_p4_activations,
        cnn_p5_activations,
        cnn_p6_activations), axis=1)

In [10]:
activations.shape

(26668, 3584)

In [11]:
y_train.shape

(26668,)

<h2>Auto Encoder</h2>

In [107]:
def get_encoder(X_train, X_test, encoding_dim):
    assert X_train.shape[1] == X_test.shape[1]
    shape = X_train.shape[1]
    validation_split = .15

    input_img = Input(shape=(shape,))
    encoded = Dense(encoding_dim, activation='sigmoid')(input_img)
    decoded = Dense(shape, activation='sigmoid')(encoded)

    autoencoder = Model(input=input_img, output=decoded)
    encoder = Model(input=input_img, output=encoded)

    # sgd = SGD(lr=.01, decay=99.9) #lr=0.01->0.00001
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
    
    autoencoder.fit(X_train, X_train,
                nb_epoch=10,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))
    return encoder
    

In [108]:
x_train = activations[:int(activations.shape[0] * (1 - validation_split))]
x_test = activations[int(activations.shape[0] * (1 - validation_split)):]
X_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
X_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [109]:
encoder = get_encoder(X_train, X_test, 2048)

Train on 22667 samples, validate on 4001 samples
Epoch 1/10
22667/22667 [==============================] - 3s - loss: -13.0636 - acc: 0.0081 - val_loss: -21.0825 - val_acc: 0.0155
Epoch 2/10
22667/22667 [==============================] - 3s - loss: -24.2497 - acc: 0.0235 - val_loss: -26.9902 - val_acc: 0.0312
Epoch 3/10
22667/22667 [==============================] - 3s - loss: -28.7776 - acc: 0.0346 - val_loss: -30.5081 - val_acc: 0.0400
Epoch 4/10
22667/22667 [==============================] - 3s - loss: -31.7733 - acc: 0.0412 - val_loss: -33.0091 - val_acc: 0.0442
Epoch 5/10
22667/22667 [==============================] - 3s - loss: -33.9836 - acc: 0.0486 - val_loss: -34.9423 - val_acc: 0.0517
Epoch 6/10
22667/22667 [==============================] - 3s - loss: -35.7482 - acc: 0.0562 - val_loss: -36.5340 - val_acc: 0.0627
Epoch 7/10
22667/22667 [==============================] - 3s - loss: -37.2314 - acc: 0.0635 - val_loss: -37.8963 - val_acc: 0.0637
Epoch 8/10
22667/22667 [==========

In [110]:
X_train = encoder.predict(X_train)
X_test = encoder.predict(X_test)

print X_train.shape
print X_test.shape

(22667, 2048)
(4001, 2048)


In [111]:
encoder = get_encoder(X_train, X_test, 1024)

Train on 22667 samples, validate on 4001 samples
Epoch 1/10
22667/22667 [==============================] - 2s - loss: 0.6902 - acc: 0.0000e+00 - val_loss: 0.6786 - val_acc: 0.0000e+00
Epoch 2/10
22667/22667 [==============================] - 2s - loss: 0.6695 - acc: 1.7647e-04 - val_loss: 0.6587 - val_acc: 0.0000e+00
Epoch 3/10
22667/22667 [==============================] - 2s - loss: 0.6460 - acc: 3.5294e-04 - val_loss: 0.6314 - val_acc: 2.4994e-04
Epoch 4/10
22667/22667 [==============================] - 2s - loss: 0.6160 - acc: 0.0011 - val_loss: 0.5996 - val_acc: 9.9975e-04
Epoch 5/10
22667/22667 [==============================] - 2s - loss: 0.5839 - acc: 0.0015 - val_loss: 0.5679 - val_acc: 0.0020
Epoch 6/10
22667/22667 [==============================] - 2s - loss: 0.5535 - acc: 0.0023 - val_loss: 0.5392 - val_acc: 0.0020
Epoch 7/10
22667/22667 [==============================] - 2s - loss: 0.5265 - acc: 0.0028 - val_loss: 0.5140 - val_acc: 0.0027
Epoch 8/10
22667/22667 [==========

In [112]:
X_train = encoder.predict(X_train)
X_test = encoder.predict(X_test)

print X_train.shape
print X_test.shape

(22667, 1024)
(4001, 1024)


In [113]:
encoder = get_encoder(X_train, X_test, 512)

Train on 22667 samples, validate on 4001 samples
Epoch 1/10
22667/22667 [==============================] - 1s - loss: 0.6923 - acc: 0.0014 - val_loss: 0.6844 - val_acc: 2.4994e-04
Epoch 2/10
22667/22667 [==============================] - 1s - loss: 0.6798 - acc: 0.0011 - val_loss: 0.6741 - val_acc: 0.0020
Epoch 3/10
22667/22667 [==============================] - 1s - loss: 0.6674 - acc: 0.0025 - val_loss: 0.6594 - val_acc: 0.0042
Epoch 4/10
22667/22667 [==============================] - 1s - loss: 0.6507 - acc: 0.0048 - val_loss: 0.6411 - val_acc: 0.0067
Epoch 5/10
22667/22667 [==============================] - 1s - loss: 0.6318 - acc: 0.0086 - val_loss: 0.6222 - val_acc: 0.0097
Epoch 6/10
22667/22667 [==============================] - 1s - loss: 0.6136 - acc: 0.0142 - val_loss: 0.6050 - val_acc: 0.0172
Epoch 7/10
22667/22667 [==============================] - 1s - loss: 0.5976 - acc: 0.0215 - val_loss: 0.5902 - val_acc: 0.0275
Epoch 8/10
22667/22667 [==============================] - 

In [114]:
X_train = encoder.predict(X_train)
X_test = encoder.predict(X_test)

print X_train.shape
print X_test.shape

(22667, 512)
(4001, 512)
